In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import time

In [2]:
ee.Initialize()

In [3]:
def get_ssm_and_timeseries(start_date, end_date, roi, chunk_size_days=30):
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(roi)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))

    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')

    avg_vv = s1_collection.mean().select('VV')
    urban_mask = avg_vv.gt(-6)
    water_mask = avg_vv.lt(-17)

    def calculate_soil_moisture(image):
        sensitivity = wet_index.subtract(dry_index)
        mv = image.select('VV').subtract(dry_index).divide(sensitivity)
        mv = mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
        return image.addBands(mv.rename('SoilMoisture'))

    soil_moisture = s1_collection.map(calculate_soil_moisture)

    def date_chunks(start, end, chunk_size):
        current = start
        while current < end:
            chunk_end = min(current + timedelta(days=chunk_size), end)
            yield current, chunk_end
            current = chunk_end

    time_series_data = []
    total_chunks = sum(1 for _ in date_chunks(
        datetime.strptime(start_date, '%Y-%m-%d'),
        datetime.strptime(end_date, '%Y-%m-%d'),
        chunk_size_days))

    for chunk_start, chunk_end in tqdm(date_chunks(
            datetime.strptime(start_date, '%Y-%m-%d'),
            datetime.strptime(end_date, '%Y-%m-%d'),
            chunk_size_days), total=total_chunks, desc="Processing chunks"):

        # time.sleep(2)  # Add delay to avoid rate limits

        chunk_collection = soil_moisture.filterDate(
            chunk_start.strftime('%Y-%m-%d'),
            chunk_end.strftime('%Y-%m-%d'))

        def extract_mean_soil_moisture(image):
            mean_value = image.select('SoilMoisture').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=roi,
                scale=10,
                maxPixels=1e9
            ).get('SoilMoisture')
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
            return ee.Feature(None, {'date': date, 'mean_soil_moisture': mean_value})

        try:
            chunk_data = chunk_collection.map(extract_mean_soil_moisture).getInfo()
        except ee.EEException as e:
            print(f"Error processing chunk {chunk_start} to {chunk_end}: {e}")
            continue  # Skip to the next chunk on error

        data = [{'date': feature['properties']['date'], 
                 'mean_soil_moisture': feature['properties']['mean_soil_moisture']}
                for feature in chunk_data['features']]
        time_series_data.extend(data)

    df = pd.DataFrame(time_series_data)
    latest_image = soil_moisture.sort('system:time_start', False).first().clip(roi)
    return latest_image, df

In [ ]:
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')
start_date = '2023-04-01'
end_date = '2023-04-30'
chunk_size_days = 20

latest_image, timeseries_df = get_ssm_and_timeseries(start_date, end_date, roi)

Map = geemap.Map()
blues_palette = ['#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', 
                 '#4292c6', '#2171b5', '#08519c', '#08306b']

Map.addLayer(latest_image.select('SoilMoisture'), 
             {'min': 0, 'max': 1, 'palette': blues_palette}, 
             'Soil Moisture (Blues Scale)')
Map.centerObject(roi, 10)
Map



Processing chunks:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def get_ssm_and_timeseries(start_date, end_date, roi):
    # Fetch Sentinel-1 collection and filter it by date and ROI
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(roi)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV', 'VH']))

    # Calculate Wet and Dry Indices
    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')

    # Define urban and water masks
    avg_vv = s1_collection.mean().select('VV')
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB

    # Compute Soil Moisture with masks applied
    def calculate_soil_moisture(image):
        sensitivity = wet_index.subtract(dry_index)
        mv = image.select('VV').subtract(dry_index).divide(sensitivity)
        mv = mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
        return image.addBands(mv.rename('SoilMoisture'))

    soil_moisture = s1_collection.map(calculate_soil_moisture)

    # Get the latest soil moisture image
    latest_image = soil_moisture.sort('system:time_start', False).first().clip(roi)

    # Generate a time series of mean soil moisture
    def extract_mean_soil_moisture(image):
        mean_value = image.select('SoilMoisture').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get('SoilMoisture')
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        return ee.Feature(None, {'date': date, 'mean_soil_moisture': mean_value})

    time_series = soil_moisture.map(extract_mean_soil_moisture).getInfo()

    # Convert the time series to a pandas DataFrame
    data = [{'date': feature['properties']['date'], 
             'mean_soil_moisture': feature['properties']['mean_soil_moisture']}
            for feature in time_series['features']]
    df = pd.DataFrame(data)

    # Return the latest image and the DataFrame with the time series
    return latest_image, df


In [ ]:

# Example usage:
roi = ee.Geometry.Polygon([[
    [-122.522, 37.733],
    [-122.355, 37.733],
    [-122.355, 37.812],
    [-122.522, 37.812]
]])

start_date = '2023-01-08'
end_date = '2023-01-20'

latest_image, timeseries_df = get_ssm_and_timeseries(start_date, end_date, roi)

# Create a map and apply a linear colour scale for soil moisture
Map = geemap.Map()
palette = ['#0000ff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000']  # Blue to Red linear scale

Map.addLayer(latest_image.select('SoilMoisture'), 
             {'min': 0, 'max': 1, 'palette': palette}, 
             'Soil Moisture (Linear Scale)')
Map.centerObject(roi, 10)
Map


In [8]:
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')
start_date = '2021-01-01'
end_date = '2023-04-30'
chunk_size_days = 20

latest_image, timeseries_df = get_ssm_and_timeseries(start_date, end_date, roi, chunk_size_days)

Map = geemap.Map()
blues_palette = ['#f7fbff', '#deebf7', '#c6dbef', '#9ecae1', '#6baed6', 
                 '#4292c6', '#2171b5', '#08519c', '#08306b']

Map.addLayer(latest_image.select('SoilMoisture'), 
             {'min': 0, 'max': 1, 'palette': blues_palette}, 
             'Soil Moisture (Blues Scale)')
Map.centerObject(roi, 10)
Map


Processing chunks:   5%|▍         | 2/43 [00:07<02:34,  3.77s/it]

In [11]:
def calculate_soil_moisture(image, wet_index, dry_index, water_mask, urban_mask):
    """Add the SoilMoisture band to an image."""
    sensitivity = wet_index.subtract(dry_index)
    mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    mv = mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    return image.addBands(mv.rename('SoilMoisture'))  # Add SoilMoisture band

def extract_soil_moisture(image, feature, feature_id_column='CNLNM_ID'):
    """Extract mean soil moisture for a given feature."""
    try:
        mean_value = image.select('SoilMoisture').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=10,
            maxPixels=1e9
        ).getInfo()['SoilMoisture']

        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        feature_id = feature.get(feature_id_column).getInfo()  # Use the correct ID column
        return {'feature_id': feature_id, 'date': date, 'mean_soil_moisture': mean_value}

    except Exception as e:
        print(f"Error processing feature: {e}")
        return None

def get_ssm_timeseries_for_features(start_date, end_date, roi):
    # Fetch Sentinel-1 collection and filter by date and ROI
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(roi)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))

    # Calculate Wet and Dry Indices
    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')

    # Define urban and water masks
    avg_vv = s1_collection.mean().select('VV')
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB

    # Apply soil moisture calculation to each image
    soil_moisture = s1_collection.map(
        lambda img: calculate_soil_moisture(img, wet_index, dry_index, water_mask, urban_mask)
    )

    # Extract individual features from the ROI
    features = roi.getInfo()['features']

    # Collect time series data for each feature
    time_series_data = []

    for feature_info in features:
        feature = ee.Feature(feature_info)

        # Iterate through each image in the collection
        for image_info in soil_moisture.toList(soil_moisture.size()).getInfo():
            image = ee.Image(image_info['id'])

            # Extract soil moisture data for the current feature and image
            result = extract_soil_moisture(image, feature, 'CNLNM_ID')
            if result:
                time_series_data.append(result)

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(time_series_data)

    # Convert the features into a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(roi.getInfo()['features'])

    # Merge the time series data with the GeoDataFrame
    gdf = gdf.merge(df, left_on='CNLNM_ID', right_on='feature_id')

    return gdf


In [12]:
# roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')
roi = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')

start_date = '2023-01-01'
end_date = '2023-04-30'

gdf = get_ssm_timeseries_for_features(start_date, end_date, roi)


Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH, angle]
Error processing feature: Image.select: Band pattern 'SoilMoisture' did not match any bands. Available bands: [VV, VH,

KeyboardInterrupt: 

In [5]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import folium

In [4]:
ee.Initialize()

In [ ]:
# Define start and end dates
start_date = '2023-01-08'
end_date = '2023-01-20'

# Define Area of Interest (replace with coordinates)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Load Sentinel-1 Image Collection
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(roi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .select(['VV']))

# Calculate Wet and Dry Indices
wet_index = s1_collection.max().select('VV')
dry_index = s1_collection.min().select('VV')

# Define urban and water masks
avg_vv = s1_collection.mean().select('VV')
urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB

# Compute Soil Moisture with Masks Applied
def calculate_soil_moisture(image):
    sensitivity = wet_index.subtract(dry_index)
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    print(f'Mean SSM: ')
    return image.addBands(Mv.rename('SoilMoisture'))

soil_moisture = s1_collection.map(calculate_soil_moisture)
final_image = soil_moisture.median().clip(roi)

# Compute Mean Soil Moisture
mean_soil_moisture = final_image.select('SoilMoisture').reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=roi,
    scale=10,
    maxPixels=1e9
)

# Print Mean Soil Moisture
print('Mean Soil Moisture:', mean_soil_moisture.getInfo())

# Display Layers
Map = geemap.Map()

# Add Soil Moisture layer
soil_moisture_vis_params = {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'brown']}
Map.add_ee_layer(final_image.select('SoilMoisture'), soil_moisture_vis_params, 'Soil Moisture')
Map.centerObject(roi, zoom=10) 

Map

Mean Soil Moisture: {'SoilMoisture': 0.5014818317817842}


Map(center=[25.858832414626484, 89.0065953822235], controls=(WidgetControl(options=['position', 'transparent_b…

In [8]:
import ee
import datetime
import pandas as pd
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define parameters
start_date = '2023-01-08'
end_date = '2023-12-31'  # Define the desired end date
chunk_size_days = 30  # Number of days for each chunk

# Load Area of Interest (replace with coordinates in shapefile)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Convert string dates to datetime objects
start_date_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_date_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d")

# Define urban and water mask function
def get_masks(s1_collection):
    avg_vv = s1_collection.mean().select('VV')
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
    return urban_mask, water_mask

# Function to calculate mean soil moisture for a given period
def get_mean_soil_moisture_for_period(start, end):
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(roi)
                     .filterDate(start, end)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))
    
    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')
    
    urban_mask, water_mask = get_masks(s1_collection)
    
    def calculate_soil_moisture(image):
        sensitivity = wet_index.subtract(dry_index)
        Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
        Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
        return image.addBands(Mv.rename('SoilMoisture'))
    
    soil_moisture = s1_collection.map(calculate_soil_moisture)
    final_image = soil_moisture.median().clip(roi)
    
    mean_soil_moisture = final_image.select('SoilMoisture').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    )
    
    return mean_soil_moisture.getInfo().get('SoilMoisture', None)

# Generate time series in chunks
results = []
current_start = start_date_dt

while current_start < end_date_dt:
    current_end = min(current_start + datetime.timedelta(days=chunk_size_days), end_date_dt)
    start_str = current_start.strftime("%Y-%m-%d")
    end_str = current_end.strftime("%Y-%m-%d")
    mean_moisture = get_mean_soil_moisture_for_period(start_str, end_str)
    
    # Record the midpoint date of the chunk for the time series
    midpoint = current_start + (current_end - current_start) / 2
    results.append({'Date': midpoint, 'MeanSoilMoisture': mean_moisture})
    
    # Move to the next chunk
    current_start = current_end + datetime.timedelta(days=1)

# Convert results to a DataFrame for time series analysis
df = pd.DataFrame(results)
print(df)

# Display Layers
Map = geemap.Map()
soil_moisture_vis_params = {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'brown']}
Map.add_ee_layer(final_image.select('SoilMoisture'), soil_moisture_vis_params, 'Soil Moisture')
Map.centerObject(roi, zoom=10)

Map


         Date  MeanSoilMoisture
0  2023-01-23          0.516947
1  2023-02-23          0.524481
2  2023-03-26          0.516647
3  2023-04-26          0.517231
4  2023-05-27          0.518573
5  2023-06-27          0.530135
6  2023-07-28          0.526595
7  2023-08-28          0.538946
8  2023-09-28          0.535542
9  2023-10-29          0.506544
10 2023-11-29          0.512097
11 2023-12-23          0.490766


Map(center=[25.858832414626484, 89.0065953822235], controls=(WidgetControl(options=['position', 'transparent_b…

In [2]:
import ee
import datetime
import numpy as np
import pandas as pd
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define parameters
start_date = '2023-01-08'
end_date = '2023-12-31'
chunk_size_days = 30  # Length of each chunk in days

# Load Area of Interest (replace with coordinates in shapefile)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Convert string dates to datetime objects and create date range list
dates_list = pd.date_range(start=start_date, end=end_date, freq='11D')
# Split the date range into 30-day chunks
no_chunks = int(len(dates_list) / chunk_size_days)
dates_list_chunks = np.array_split(dates_list, no_chunks) if no_chunks > 0 else [dates_list]

# Function to calculate soil moisture for each image within a chunk
def calculate_soil_moisture_per_image(image, wet_index, dry_index, urban_mask, water_mask):
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean_soil_moisture = Mv.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('VV')
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_soil_moisture})

# Creating an empty list to hold all results
all_results = []

print(f'Date list chunk: {dates_list_chunks}')

# Process each 30-day chunk
for chunk in dates_list_chunks:
    # Define start and end of the current chunk
    chunk_start = chunk[0].strftime('%Y-%m-%d')
    chunk_end = chunk[-1].strftime('%Y-%m-%d')
    print(f'Chunk start Date: {chunk_start}')
    print(f'Chunk end Date: {chunk_end}')
    # Load Sentinel-1 Image Collection for the current chunk
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(roi)
                     .filterDate(chunk_start, chunk_end)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))
    
    # Calculate wet and dry indices for the current chunk
    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')
    print(f'Count of images: {s1_collection.size().getInfo()}')
    # Define urban and water masks
    avg_vv = s1_collection.mean().select('VV')
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
    
    # Calculate SSM for each image in the chunk
    soil_moisture_features = s1_collection.map(
        lambda image: calculate_soil_moisture_per_image(image, wet_index, dry_index, urban_mask, water_mask)
    )

    # Retrieve SSM values from each image in the chunk and add to the results list
    ssm_features = soil_moisture_features.getInfo()
    # print(ssm_features)
    chunk_data = [{'Date': f['properties']['date'], 'MeanSoilMoisture': f['properties']['MeanSoilMoisture']} for f in ssm_features['features']]
    
    # # Append chunk data to all results
    all_results.extend(chunk_data)

# # Convert results to a DataFrame for the time series
df = pd.DataFrame(all_results)
print(df)


Date list chunk: [DatetimeIndex(['2023-01-08', '2023-01-19', '2023-01-30', '2023-02-10',
               '2023-02-21', '2023-03-04', '2023-03-15', '2023-03-26',
               '2023-04-06', '2023-04-17', '2023-04-28', '2023-05-09',
               '2023-05-20', '2023-05-31', '2023-06-11', '2023-06-22',
               '2023-07-03', '2023-07-14', '2023-07-25', '2023-08-05',
               '2023-08-16', '2023-08-27', '2023-09-07', '2023-09-18',
               '2023-09-29', '2023-10-10', '2023-10-21', '2023-11-01',
               '2023-11-12', '2023-11-23', '2023-12-04', '2023-12-15',
               '2023-12-26'],
              dtype='datetime64[ns]', freq=None)]
Chunk start Date: 2023-01-08
Chunk end Date: 2023-12-26
Count of images: 110


EEException: Computation timed out.

In [ ]:
import ee
import pandas as pd
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define parameters
start_date = '2023-01-08'
end_date = '2023-12-31'

# Load Area of Interest (replace with coordinates in shapefile)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Load Sentinel-1 Image Collection
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(roi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .select(['VV']))

# Calculate wet and dry indices for the entire period
wet_index = s1_collection.max().select('VV')
dry_index = s1_collection.min().select('VV')

# Define urban and water masks based on the entire period
avg_vv = s1_collection.mean().select('VV')
urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB

# Function to calculate soil moisture for each image
def calculate_soil_moisture_per_image(image):
    
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    
    # Extract the date and mean soil moisture for each image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean_soil_moisture = Mv.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('SoilMoisture')
    
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_soil_moisture})

# Map the function over each image in the collection and get results
soil_moisture_features = s1_collection.map(calculate_soil_moisture_per_image)

# Convert results to a list of dictionaries
ssm_features = soil_moisture_features.getInfo()
ssm_data = [{'Date': f['properties']['date'], 'MeanSoilMoisture': f['properties']['MeanSoilMoisture']} for f in ssm_features['features']]

# Convert to DataFrame for analysis
df = pd.DataFrame(ssm_data)
print(df)


EEException: Error in map(ID=S1A_IW_GRDH_1SDV_20230131T000344_20230131T000409_047020_05A3D7_F39E):
Dictionary.get: Dictionary does not contain key: 'SoilMoisture'.

In [ ]:

# Define parameters
start_date = '2023-01-08'
end_date = '2023-12-20'

# Load Area of Interest (replace with coordinates in shapefile)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Load Sentinel-1 Image Collection
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                 .filterBounds(roi)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .select(['VV']))

# Calculate Wet and Dry Indices for the entire period
wet_index = s1_collection.max().select('VV')
dry_index = s1_collection.min().select('VV')
# Define urban and water masks
avg_vv = s1_collection.mean().select('VV')
urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB

# Function to calculate soil moisture for each image and store it in a feature collection
def calculate_soil_moisture(image):
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    # Calculate mean soil moisture for each image
    mean_ssm = Mv.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('VV')
    # print(f'mean_ssm: {mean_ssm.getInfo()}')
    
    # Get the date of the image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_ssm})

first_image = s1_collection.first()
# test_image = calculate_soil_moisture(first_image)
# print("Test Image with Soil Moisture Band:", test_image.getInfo())
# # Map the function over each image in the collection to create a time series of mean SSM
soil_moisture_features = s1_collection.map(calculate_soil_moisture)

# # Convert to a list of dictionaries and then to a DataFrame
ssm_features = soil_moisture_features.getInfo()
ssm_data = [{'Date': f['properties']['date'], 'MeanSoilMoisture': f['properties']['MeanSoilMoisture']} for f in ssm_features['features']]

# Convert to DataFrame
df = pd.DataFrame(ssm_data)
print(df)

In [3]:
import ee
import pandas as pd
import datetime
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define parameters
start_date = '2023-01-01'
end_date = '2023-12-31'
chunk_size_days = 30  # Each chunk is 30 days long

# Load Area of Interest (replace with coordinates in shapefile)
roi = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')

# Function to calculate soil moisture for each image
def calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask):
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    
    # Calculate mean soil moisture for each image
    mean_ssm = Mv.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('VV')
    
    # Get the date of the image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_ssm})

# Generate a list of start and end dates for each 30-day chunk
start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
dates = pd.date_range(start=start_date_dt, end=end_date_dt, freq=f'{chunk_size_days}D')

# Initialize an empty list to store results
all_results = []

# Process each 30-day chunk
for i in range(len(dates) - 1):
    chunk_start = dates[i].strftime('%Y-%m-%d')
    chunk_end = dates[i + 1].strftime('%Y-%m-%d')
    
    # Load Sentinel-1 Image Collection for the current chunk
    s1_collection_chunk = (ee.ImageCollection('COPERNICUS/S1_GRD')
                           .filterBounds(roi)
                           .filterDate(chunk_start, chunk_end)
                           .filter(ee.Filter.eq('instrumentMode', 'IW'))
                           .select(['VV']))
    
    # Calculate wet and dry indices for the current chunk
    wet_index = s1_collection_chunk.max().select('VV')
    dry_index = s1_collection_chunk.min().select('VV')
    
    # Define urban and water masks
    avg_vv = s1_collection_chunk.mean().select('VV')
    urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
    water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
    
    # Calculate SSM for each image in the chunk
    soil_moisture_features = s1_collection_chunk.map(
        lambda image: calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask)
    )
    
    # Retrieve SSM values from each image in the chunk and add to the results list
    ssm_features = soil_moisture_features.getInfo()
    chunk_data = [{'Date': f['properties']['date'], 'MeanSoilMoisture': f['properties']['MeanSoilMoisture']} for f in ssm_features['features']]
    
    # Append chunk data to all results
    all_results.extend(chunk_data)

# Convert results to a DataFrame for the time series
df = pd.DataFrame(all_results)
print(df)


           Date  MeanSoilMoisture
0    2023-01-01          0.405761
1    2023-01-04          0.358111
2    2023-01-07          0.619634
3    2023-01-11          0.639498
4    2023-01-13          0.395745
..          ...               ...
108  2023-12-09          0.669300
109  2023-12-13          0.614693
110  2023-12-18          0.288069
111  2023-12-21          0.529516
112  2023-12-25          0.559771

[113 rows x 2 columns]


In [6]:
df['MeanSoilMoisture'].describe()

count    113.000000
mean       0.498050
std        0.112780
min        0.276829
25%        0.391886
50%        0.529516
75%        0.589341
max        0.706832
Name: MeanSoilMoisture, dtype: float64

In [10]:
import ee
import pandas as pd
import datetime
import geemap
import geopandas as gpd

# Initialize the Earth Engine API
ee.Initialize()

# Define parameters
start_date = '2023-01-01'
end_date = '2023-12-31'
chunk_size_days = 30  # Each chunk is 30 days long

# Load the shapefile and convert to a feature collection
# roi_fc = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')
roi_fc = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')

# Function to calculate soil moisture for each image within a given region
def calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask, region):
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    
    # Calculate mean soil moisture for each image in the specified region
    mean_ssm = Mv.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region.geometry(),
        scale=10,
        maxPixels=1e9
    ).get('VV')
    
    # Get the date of the image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_ssm, 'RegionID': region.get('CNLNMID')})

# Generate a list of start and end dates for each 30-day chunk
start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
dates = pd.date_range(start=start_date_dt, end=end_date_dt, freq=f'{chunk_size_days}D')

# Initialize a list to store dataframes for each region
region_dataframes = []

# Iterate over each region (feature) in the ROI feature collection
for region in roi_fc.toList(roi_fc.size()).getInfo():
    region_feature = ee.Feature(region)
    region_id = region_feature.get('CNLNM_ID').getInfo()  # Use 'CNLNMID' as the region ID

    # Process each 30-day chunk for the current region
    all_results = []
    for i in range(len(dates) - 1):
        chunk_start = dates[i].strftime('%Y-%m-%d')
        chunk_end = dates[i + 1].strftime('%Y-%m-%d')
        
        # Load Sentinel-1 Image Collection for the current chunk
        s1_collection_chunk = (ee.ImageCollection('COPERNICUS/S1_GRD')
                               .filterBounds(region_feature.geometry())
                               .filterDate(chunk_start, chunk_end)
                               .filter(ee.Filter.eq('instrumentMode', 'IW'))
                               .select(['VV']))
        
        # Calculate wet and dry indices for the current chunk
        wet_index = s1_collection_chunk.max().select('VV')
        dry_index = s1_collection_chunk.min().select('VV')
        
        # Define urban and water masks
        avg_vv = s1_collection_chunk.mean().select('VV')
        urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
        water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
        
        # Calculate SSM for each image in the chunk for the current region
        soil_moisture_features = s1_collection_chunk.map(
            lambda image: calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask, region_feature)
        )
        
        # Retrieve SSM values from each image in the chunk
        ssm_features = soil_moisture_features.getInfo()
        chunk_data = [{'Date': f['properties']['date'],
                       'MeanSoilMoisture': f['properties']['MeanSoilMoisture'],
                       'RegionID': f['properties']['RegionID']} for f in ssm_features['features']]
        
        # Append chunk data to all results for the region
        all_results.extend(chunk_data)
    
    # Create a DataFrame for the current region and add it to the list
    region_df = pd.DataFrame(all_results)
    region_df['RegionID'] = region_id  # Ensure RegionID is present in DataFrame
    region_dataframes.append(region_df)

# Combine all regional dataframes into a single GeoDataFrame
final_df = pd.concat(region_dataframes, ignore_index=True)
gdf = gpd.GeoDataFrame(final_df)

print(gdf)


             Date  MeanSoilMoisture           RegionID
0      2023-01-04          0.376385           T1S1B_95
1      2023-01-07          0.633811           T1S1B_95
2      2023-01-11          0.624130           T1S1B_95
3      2023-01-16          0.357811           T1S1B_95
4      2023-01-19          0.606388           T1S1B_95
...           ...               ...                ...
14135  2023-12-09          0.739723  Dinajpur_Canal_69
14136  2023-12-13          0.530822  Dinajpur_Canal_69
14137  2023-12-18          0.261108  Dinajpur_Canal_69
14138  2023-12-21          0.511555  Dinajpur_Canal_69
14139  2023-12-25          0.428580  Dinajpur_Canal_69

[14140 rows x 3 columns]


In [ ]:
gdf['Date'] = pd.to_datetime(gdf['Date'], format = '%Y-%m-%d')

TypeError: 'Series' object is not callable

In [26]:
gdf.to_csv('../Paper_Revision/SSM_Sentinel1/SSM_S1.csv', index = False)